In [ ]:
# ### 코랩용
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/Othercomputers/내 노트북/gdrive/dppos/flat')
# !pip uninstall -y gym
# !pip install gymnasium stable-baselines3


텐서보드 log 확인

```
tensorboard --logdir ./logs/
```

[http://localhost:6006](http://localhost:6006)

In [ ]:
import time
from drivingppo.common import set_seed
from drivingppo.environment import WorldEnv
from drivingppo.model import train_start, train_resume, evaluate
from world_samples import gen_0, gen_1, gen_2

import numpy as np
from IPython.display import clear_output

prefix = 'flat'
t0 = time.strftime('%y%m%d-%H%M')

def a(seed):
    run_name = f'{prefix}-{t0}-{seed}'

    set_seed(seed)
    model = train_start(
        save_path=f'{prefix}-{seed}--0',
        gen_env=lambda: WorldEnv(world_generator=gen_0, time_gain_per_waypoint_rate=2_000, time_gain_limit=30_000),
        tb_log=True,
        run_name=run_name,
        steps=4_0000,
        lr=9e-4,
        gamma=0.95,
        ent_coef=0.02,
        seed=seed
    )

    passed = False  # 테스트를 통과해야 다음 단계로 간다.
    while not passed:
        set_seed(seed)
        model = train_resume(
            model=model,
            save_path=f'{prefix}-{seed}--1',
            gen_env=lambda: WorldEnv(world_generator=gen_1, time_gain_per_waypoint_rate=1_000),
            tb_log=True,
            run_name=run_name,
            steps=10_0000,
            lr=3e-4,
            gamma=0.99,
            ent_coef=0.02,
            seed=seed
        )
        metrics = evaluate(model, gen_1, csv_path='', episode_num=100)
        if np.mean(metrics['rewards/1.wPoint']) > 0.2:
            passed = True
        else:
            print('=== 1단계 통과 못함. 다시')

    set_seed(seed)
    model = train_resume(
        model=model,
        save_path=f'{prefix}-{seed}--2',
        gen_env=lambda: WorldEnv(world_generator=gen_2),
        tb_log=True,
        run_name=run_name,
        steps=80_0000,
        lr=3e-4,
        gamma=0.996,
        seed=seed
    )

    set_seed(seed)
    model = train_resume(
        model=model,
        save_path=f'{prefix}-{seed}--3',
        gen_env=lambda: WorldEnv(world_generator=gen_2),
        tb_log=True,
        run_name=run_name,
        save_freq=10_00000,
        steps=100_0000,
        lr=1e-4,
        gamma=0.996,
        seed=seed
    )
    clear_output(wait=True)
    evaluate(f'{prefix}-{seed}--3', gen_1, episode_num=100, csv_path='', verbose=True)

a(100)
a(200)
a(300)
a(400)
a(500)

| `log_std` | 실제 표준편차 σ | 탐색 수준 | 비고 |
| --- | --- | --- | --- |
| 0 | 1.0 | 매우 높음 | 행동 공간 전체를 휘젓고 다니는 수준. 초기 학습 시 강한 탐색이 필요할 때 사용. |
| -0.5 | ≈ 0.6 | 높음 | 행동의 변동폭이 큼. 기존 정책이 굳었을 때 강제 리셋용으로 권장되는 값. |
| -1 | ≈ 0.37 | 보통 | 안정적인 주행과 탐색이 공존하는 수준. |
| -2 | ≈ 0.13 | 낮음 | 어느 정도 최적화가 진행된 상태. 정밀한 제어를 시작함. |
| \~-3.0 | \< 0.05 | 거의 없음 | 수렴 완료 단계. 거의 결정론적(Deterministic)인 행동을 보임. |